In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from colormath.color_objects import sRGBColor, XYZColor
from colormath.color_conversions import convert_color
from scipy.optimize import minimize
import scipy.interpolate as intp
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
MF = np.genfromtxt("CMF_5nm.csv", delimiter=',') 
print(MF.shape)
CMF = MF[:, 1:4]

(89, 4)


In [3]:
# make unit vector array
coeffs = np.linspace(0, 1, 30)
ucmf = [c/np.linalg.norm(c) for c in CMF]
c1 = CMF[-1]/np.linalg.norm(CMF[-1])
c0 = CMF[0]/np.linalg.norm(CMF[0])
mixture = np.array([c*c1 +(1-c)*c0 for c in coeffs])
CMFA = np.concatenate((ucmf, mixture))

In [4]:
def clip2rgb(xyz):
    if type(xyz) is not XYZColor:
        xyz = XYZColor(xyz[0], xyz[1], xyz[2])
    crgb = convert_color(xyz, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    return [r, g, b]

In [5]:
def g_euc(r1, r2):
    return np.sqrt((r1[0]-r2[0])**2 + (r1[1]-r2[1])**2 + (r1[2]-r2[2])**2)

In [6]:
# plot delta colors and lines
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlim(sum(CMFA[:,0]))
ax.set_ylim(sum(CMFA[:,1]))
ax.set_zlim(sum(CMFA[:,2]))
# line info
xvec = np.linspace(0, sum(CMFA[:,0]), 100)

L = CMFA.shape[0]
C0 = []
C1 = []
C2 = []
RGB = []
for O in CMFA:
    C = XYZColor(O[0], O[1], O[2])
    rgb = clip2rgb(C)
    RGB.append(rgb)
    yvec = (O[1]/O[0])*(xvec-O[0]) + O[1]
    zvec = (O[2]/O[0])*(xvec-O[0]) + O[2]
    ax.plot(xvec, yvec, zvec, color=rgb)

In [7]:
# plot gap colors and lines
C0 = []
C1 = []
C2 = []
RGB = []
for O in CMFA:
    [w1, w2, w3] = sum(CMFA)
    C = XYZColor(1-O[0], 1-O[1], 1-O[2])
    rgb = clip2rgb(C)
    RGB.append(rgb)
    yvec = (O[1]/O[0])*(xvec-w1) + w2
    zvec = (O[2]/O[0])*(xvec-w1) + w3
    ax.plot(xvec, yvec, zvec, color=rgb)

In [8]:
def XYZ2xyY(xyz):
    if type(xyz) is XYZColor:
        xyz = [xyz.xyz_x, xyz.xyz_y, xyz.xyz_z]
    r = sum(xyz)
    x = xyz[0]/r
    y = xyz[1]/r
    return (x, y, xyz[1])

In [9]:
# plot chromaticity diagram in xyY and lines with white point
L = CMFA.shape[0]
abeam = np.ones(L)
fig = plt.figure()
plt.ylim(0,1)
white = np.dot(abeam, CMFA)
wtpoint = white/np.linalg.norm(white)
(wx, wy, _) = XYZ2xyY(wtpoint)
plt.scatter(wx, wy, color=(0.9, 0.9, 0.9))
xvec = np.linspace(0, 1, 10)
for i, c in enumerate(CMFA):
    rgb = clip2rgb(c)
    xyY = XYZ2xyY(c)
    m = (wy - xyY[1])/(wx - xyY[0])
    yvec = m*xvec + xyY[1] - m*xyY[0]
    plt.plot(xvec, yvec, color=rgb)
    plt.scatter(xyY[0], xyY[1], color=rgb)
    plt.text(xyY[0], xyY[1], i)
print(len(CMFA))

119


In [10]:
def g_line2point(ln, pt):
    # distance between point and line
    # line ~ (a,b,c) coefficients of ax + by + c = 0
    # point ~ (x0, y0)
    return abs(ln[0]*pt[0] + ln[1]*pt[1] + ln[2])/np.sqrt(ln[0]**2 + ln[1]**2)

def comp(i, CMF, wtpt):
    """Return complementary wavelength-index of wl-index i
    assuming 5-nm CMF, returns -1 if i has no complementary"""
    [wx, wy, _] = XYZ2xyY(wtpt)
    # [0] find comp0 and comp1 for given white point
    [x0, y0, _] = XYZ2xyY(CMF[0])
    slope = (wy - y0)/(wx - x0)
    a = -slope
    b = 1
    c = slope*x0-y0
    line = (a, b, c)
    dvec = []
    for c in CMF[int(len(CMF)/4):]:
        xyY = XYZ2xyY(c)
        dvec.append(g_line2point(line, xyY[0:2]))
    comp0 = np.argmin(np.array(dvec)) + int(len(CMF)/4)
    print("comp0 ", comp0)
    
    [xf, yf, _] = XYZ2xyY(CMF[len(CMF)-1])
    slope = (wy - yf)/(wx - xf)
    a = -slope
    b = 1
    c = slope*xf-yf
    line = (a, b, c)
    dvec = []
    for c in CMF[0:int(0.5*len(CMF))]:
        xyY = XYZ2xyY(c)
        dvec.append(g_line2point(line, xyY[0:2]))
    compf = np.argmin(np.array(dvec))
    print("compf ", compf)
    
    # [1] find xy coords of CMF[i] and wtpt
    [xi, yi, _] = XYZ2xyY(CMF[i])
    [wx, wy, _] = XYZ2xyY(wtpt)
    slope = (wy - yi)/(wx - xi)
    a = -slope
    b = 1
    c = slope*xi-yi
    line = (a, b, c)
    if (i > comp0) and (i < compf):
        return -1
    elif (i <= comp0):
        dvec = []
        for j in range(compf, len(CMF)):
            xyY = XYZ2xyY(CMF[j])
            dvec.append(g_line2point(line, xyY[0:2]))
        return np.argmin(np.array(dvec)) + compf
    elif (i >= compf):
        dvec = []
        for j in range(0, comp0+1):
            xyY = XYZ2xyY(CMF[j])
            dvec.append(g_line2point(line, xyY[0:2]))
        return np.argmin(np.array(dvec))

In [11]:
comp(16, CMF, sum(CMF))

comp0  36
compf  20


37

In [12]:
def dist(tvec, i, cmf):
    wtpt = sum(cmf)
    return g_euc(cmf[i]*tvec[0], -cmf[int(tvec[2])]*tvec[1]+wtpt)
result = minimize(fun=dist, x0=(0, 0, 0), args=(1, CMF))
print(result)
print(CMF[1]*497)
print(-CMF[1]*199+sum(CMF))

      fun: 28.20394832470821
 hess_inv: array([[9.45738660e+03, 3.78934847e+03, 0.00000000e+00],
       [3.78934847e+03, 1.51946180e+03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
      jac: array([-7.15255737e-07,  1.66893005e-06,  0.00000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 48
      nit: 5
     njev: 12
   status: 0
  success: True
        x: array([497.54674404, 199.3762247 ,   0.        ])
[ 4.6633346   0.52664406 22.91062648]
[20.74252319 22.39774282 13.4411904 ]


In [5]:
def character(i, wtpt, cmfa):
    """Characteristic Color of pseudo-wavelength-index i
        observe that the characteristic color is the closest point to the white point
    """
    O = cmfa[i]
    # define line in 3d space with xvec, yvec, zvec
    xvec = np.linspace(0, 60, 500)
    yvec = (O[1]/O[0])*(xvec-O[0]) + O[1]
    zvec = (O[2]/O[0])*(xvec-O[0]) + O[2]
    rvec = [np.array([xvec[i], yvec[i], zvec[i]]) for i, _ in enumerate(xvec)]
    distset = [g_euc(r, wtpt) for r in rvec]
    idx = np.argmin(distset)
    return rvec[idx]

In [6]:
L = CMFA.shape[0]
abeam = np.ones(L)
white = np.dot(abeam, CMFA)
print(character(0, white))

NameError: name 'CMFA' is not defined

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlim(sum(CMFA[:,0]))
ax.set_ylim(sum(CMFA[:,1]))
ax.set_zlim(sum(CMFA[:,2]))
for idx, _ in enumerate(CMFA):
    O = character(idx, sum(CMF))
    C = XYZColor(O[0], O[1], O[2])
    rgb = clip2rgb(C)
    ax.scatter(O[0], O[1], O[2], color=rgb)


In [ ]:
def mensurate(cmf, abeam, N): 
    # make appended unit vector array from CMF
    wtpt = cmf@abeam
    coeffs = np.linspace(0, 1, 30)
    ucmf = [c/np.linalg.norm(c) for c in cmf]
    c1 = cmf[-1]/np.linalg.norm(cmf[-1])
    c0 = cmf[0]/np.linalg.norm(cmf[0])
    mixture = np.array([c*c1 +(1-c)*c0 for c in coeffs])
    units = np.concatenate((ucmf, mixture))
    
    # find characteristic color of each unit
    xvec = np.zeros(len(units))
    yvec = np.zeros(len(units))
    zvec = np.zeros(len(units))
    for idx, _ in units:
        c = character(idx, wtpt, units) 
        xvec[idx] = c[0]
        yvec[idx] = c[1]
        zvec[idx] = c[2]
    
    
    # fit 3 pre-mensuration splines
    I = np.arange(units.size)
    x_spline = intp.make_interp_spline(I, xvec)
    y_spline = intp.make_interp_spline(I, yvec)
    z_spline = intp.make_interp_spline(I, zvec)

In [ ]:
abeam = np.ones(CMF.shape[0])
mensurate(CMF, abeam, 10)